In [2]:
import src.utilities.utils as utils 
test_data = utils.read_jsonl('data/snli_1.0/snli_1.0_test.jsonl')
train_data = utils.read_jsonl('data/snli_1.0/snli_1.0_train.jsonl')
dev_data = utils.read_jsonl('data/snli_1.0/snli_1.0_dev.jsonl')

In [1]:
1

1

In [3]:
# nums = ['2','3','4','5','6','7','8','9','10']
# special_numericals = ['some', 'many', 'all'] # TODO
# number_words = utils.plural_words_for_numbers + nums

convert_dict_plural = {
    'two':2,
    'three':3,
    'four':4,
    'five':5,
    'six':6,
    'seven':7,
    'eight':8,
    'nine':9,
    'ten':10,
    '2':2,
    '3':3,
    '4':4,
    '5':5,
    '6':6,
    '7':7,
    '8':8,
    '9':9,
    '10':10
}

def numerical_word_in_sentence(sentence):
    return [x for x in sentence.split() if x.lower() in convert_dict_plural]

def subset_with_numericals(dataset):
    result = []
    for data in dataset:
        sent1_num_words = numerical_word_in_sentence(data['sentence1'])
        sent2_num_words = numerical_word_in_sentence(data['sentence2'])

        if sent1_num_words or sent2_num_words:
            result.append(data)
            
    return result

def subset_with_same_numericals(dataset):
    result = []
    for data in dataset:
        sent1_num_words = numerical_word_in_sentence(data['sentence1'])
        sent2_num_words = numerical_word_in_sentence(data['sentence2'])

        if sent1_num_words == sent2_num_words and len(sent1_num_words) == 1: # TODO: When len(list) > 1
            result.append(data)
            
    return result

In [4]:
def subset_by_relationship_type(dataset, relation):
    result = []
    return [data for data in dataset if data['gold_label'] == relation]

In [5]:
test_data_entailment = subset_by_relationship_type(test_data, 'entailment')
train_data_entailment = subset_by_relationship_type(train_data, 'entailment')
dev_data_entailment = subset_by_relationship_type(dev_data, 'entailment')
# test_data_contradiction = subset_by_relationship_type(test_data, 'contradiction')
# test_data_neutral = subset_by_relationship_type(test_data, 'neutral')

In [6]:
len(test_data_entailment), len(train_data_entailment), len(dev_data_entailment)

(3368, 183416, 3329)

In [7]:
test_data_entailment[0]

{'annotator_labels': ['entailment',
  'entailment',
  'entailment',
  'neutral',
  'entailment'],
 'captionID': '2677109430.jpg#1',
 'gold_label': 'entailment',
 'pairID': '2677109430.jpg#1r1e',
 'sentence1': 'This church choir sings to the masses as they sing joyous songs from the book at a church.',
 'sentence1_binary_parse': '( ( This ( church choir ) ) ( ( ( sings ( to ( the masses ) ) ) ( as ( they ( ( sing ( joyous songs ) ) ( from ( ( the book ) ( at ( a church ) ) ) ) ) ) ) ) . ) )',
 'sentence1_parse': '(ROOT (S (NP (DT This) (NN church) (NN choir)) (VP (VBZ sings) (PP (TO to) (NP (DT the) (NNS masses))) (SBAR (IN as) (S (NP (PRP they)) (VP (VBP sing) (NP (JJ joyous) (NNS songs)) (PP (IN from) (NP (NP (DT the) (NN book)) (PP (IN at) (NP (DT a) (NN church))))))))) (. .)))',
 'sentence2': 'The church is filled with song.',
 'sentence2_binary_parse': '( ( The church ) ( ( is ( filled ( with song ) ) ) . ) )',
 'sentence2_parse': '(ROOT (S (NP (DT The) (NN church)) (VP (VBZ is) (V

In [8]:
# Entailment pairs that include numerical words in either one of the sentences
test_entailment_with_numericals = subset_with_numericals(test_data_entailment)
train_entailment_with_numericals = subset_with_numericals(train_data_entailment)
dev_entailment_with_numericals = subset_with_numericals(dev_data_entailment)

# Sentence with Additions

## 1. A + B = C

$[A, B]$: numerical words from premise, $[C]$: numerical word from hypothesis

Entailment: If $A + B = C$, replace $A, B, C$ such that $A + B = C $

Neutral: If $A + B = C$, replace $A, B, C$ such that $A + B < C $

In [9]:
def n2one(dataset, n):
    result = []
    for data in dataset:
        sent1_num_words = numerical_word_in_sentence(data['sentence1'])
        sent2_num_words = numerical_word_in_sentence(data['sentence2'])

        if len(sent1_num_words) == n and len(sent2_num_words) == 1:
            print((sent1_num_words, sent2_num_words))
            print(data['gold_label'])
            print(data['sentence1'])
            print(data['sentence2'])
            print()
            result.append(data)
    return result

In [28]:
def detect_num_pairs(dataset, n):
    result = []
    for data in dataset:
        sent1_num_words = numerical_word_in_sentence(data['sentence1'])
        sent2_num_words = numerical_word_in_sentence(data['sentence2'])

        if len(sent1_num_words) == n and len(sent2_num_words) == 1:
            if sum_is_equal((sent1_num_words, sent2_num_words)):
                pair_aslist = [convert_dict_plural[x.lower()] for x in sent1_num_words]
                pair = (pair_aslist[0], pair_aslist[1])
                result.append(pair)

    return set(result)
training_pairs_found = detect_num_pairs(train_entailment_with_numericals, 2)

In [29]:
training_pairs_found

{(2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 8),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 2)}

In [10]:
test_ent_two2one = n2one(test_entailment_with_numericals, 2)
train_ent_two2one = n2one(train_entailment_with_numericals, 2)
dev_ent_two2one = n2one(dev_entailment_with_numericals, 2)

(['2', '2'], ['4'])
entailment
2 women and 2 girls crossing the road
There are 4 female humans who are moving.

(['Three', 'two'], ['Five'])
entailment
Three women and two men are standing at the foot of a large staircase holding stringed instruments and a clarinet.
Five people are standing by a staircase.

(['Two', 'four'], ['Six'])
entailment
Two men stand in a lift as four others, all dressed in black and bright yellow stripes, look up on them.
Six men dressed in black and yellow uniforms are standing in a gymnasium.

(['Five', 'two'], ['Five'])
entailment
Five People are kneeling between two park cars, and one man is approaching with his arm raised into the air.
Five men are kneeling, while another man puts his hands into the air.

(['Three', 'three'], ['Three'])
entailment
Three women sit at a table, taking notes from three identical magazines.
Three women are reading and writing.

(['Three', 'two'], ['Three'])
entailment
Three men are playing soccer, two of them in blue uniforms 

entailment
Two ladies are cooking & talking and two gentlemen are talking in the opposite site of the kitchen.
Four people are in the kitchen.

(['Two', 'two'], ['Two'])
entailment
Two ladies are cooking & talking and two gentlemen are talking in the opposite site of the kitchen.
Two ladies are making a meal.

(['4', '2'], ['Four'])
entailment
Construction site with 4 men, 2 of which are on the roof, looking at something in unison.
Four humans gazing.

(['two', 'two'], ['four'])
entailment
two girls with pink streaked hair and two boys with black hair standing next to a concrete building.
There are four children in this picture, and they are all outside.

(['Two', 'two'], ['Two'])
entailment
Two referees watch as two martial artists fight.
Two people are having a fighting match.

(['Two', 'two'], ['Four'])
entailment
Two men pushing wheelbarrows, two men looking at them, they are in the desert near some block construction buildings.
Four men are in the desert.

(['Two', 'two'], ['Two']

(['Two', 'two'], ['Four'])
entailment
Two men and two women, in period costumes, performing a dance on state outdoors.
Four people performing a dance on state outdoors.

(['Three', 'two'], ['Three'])
entailment
Three men are dressed in strange costumes, two are carrying musical instruments.
Three men dressed in costume.

(['Three', 'two'], ['three'])
entailment
Three men, two in black business suits and one in a black dress shirt are reviewing notes for a presentation.
There are at least three people.

(['Four', 'two'], ['Four'])
entailment
Four boys standing outside begin to run while two others watch on.
Four boys are outside.

(['Two', 'three'], ['Five'])
entailment
Two people are feeding sheep in a field with a dog nearby and three more people looking at them.
Five people are outside near sheep.

(['Two', 'two'], ['Two'])
entailment
Two men, one of which has two prosthetic legs, are walking on pavement.
Two men are walking.

(['Two', 'six'], ['eight'])
entailment
Two rowers in yell

Three men with cameras outside.

(['three', 'two'], ['Two'])
entailment
Here are three photographers in cargo shorts, two of which are about to snap a picture.
Two photographers are getting ready to take a picture.

(['three', 'two'], ['3'])
entailment
A group of three women play the card game of Uno while two other people hold out their hands with their cards.
A group of 3 women play Uno.

(['Two', 'two'], ['Two'])
entailment
Two little boys with spoons stirring something in two metal pans.
Two little boys are holding spoons.

(['Three', 'two'], ['Three'])
entailment
Three men, two holding a shovel, are standing on side of a mountain.
Three men standing near a mountainside.

(['Two', 'two'], ['Four'])
entailment
Two girls and two boys are walking down a road with backpacks on their backs.
Four kids are walking together.

(['Three', 'two'], ['three'])
entailment
Three people sit on a couch, two with drinks in their hand, while several other people mill around in the background.
There a

Two naked boys standing on back of an ox while two other boys try to climb out of the water onto him as well.
two boys standing

(['Four', 'two'], ['Four'])
entailment
Four men are playing dominoes and two of the men are wearing baseball caps.
Four men are playing a game.

(['Three', 'two'], ['five'])
entailment
Three men and two women walk down an outdoor hallway past a man sitting on a step reading a newspaper.
There are five people walking down a hallway.

(['Three', 'two'], ['six'])
entailment
Three men and two women walk down an outdoor hallway past a man sitting on a step reading a newspaper.
There are six people in total in this scene

(['Three', 'two'], ['5'])
entailment
Three men and two women walk down an outdoor hallway past a man sitting on a step reading a newspaper.
A group of 5 is walking down a hallway.

(['three', 'three'], ['three'])
entailment
three children ride three wheelers.
children are riding three wheelers.

(['Five', 'three'], ['Five'])
entailment
Five childr

In [11]:
def get_binary_pairs(list_of_dic):
    res = []
    for each in list_of_dic:
        res.append(
            (each['sentence1_binary_parse'], each['sentence2_binary_parse'])
        )
    return res

test_bin_ent_two2one = get_binary_pairs(test_ent_two2one)
train_bin_ent_two2one = get_binary_pairs(train_ent_two2one)
dev_bin_ent_two2one = get_binary_pairs(dev_ent_two2one)

In [12]:
test_bin_ent_two2one[0]

('( ( ( 2 women ) and ) ( ( 2 girls ) ( crossing ( the road ) ) ) )',
 '( There ( ( are ( ( 4 ( female humans ) ) ( who ( are moving ) ) ) ) . ) )')

In [13]:
num_pairs_example = numerical_word_in_sentence(test_bin_ent_two2one[0][0]), numerical_word_in_sentence(test_bin_ent_two2one[0][1])
num_pairs_example

(['2', '2'], ['4'])

In [14]:
# function to check if A + B = C
def sum_is_equal(num_pairs):
    return sum([convert_dict_plural[x.lower()] for x in num_pairs[0]]) == sum([convert_dict_plural[x.lower()] for x in num_pairs[1]])

In [15]:
sum_is_equal(num_pairs_example)

True

In [20]:

all_possible_pairs = []
for each in  entailment_sum_dict.values():
    all_possible_pairs += each

In [30]:
already_found_in_training = {(2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 8),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 2)}

In [64]:
temp = set()
for pair in already_found_in_training:
    temp.add((pair[1],pair[0]))
already_found_in_training_add_reverses = already_found_in_training.union(temp)

In [66]:
already_found_in_training_add_reverses

{(2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 8),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 2),
 (4, 3),
 (5, 2),
 (6, 2),
 (8, 2)}

In [32]:
all_possible_pairs = set([(x[0], x[1]) for x in all_possible_pairs])

In [69]:
all_possible_pairs - already_found_in_training_add_reverses

{(2, 7),
 (3, 5),
 (3, 6),
 (3, 7),
 (4, 4),
 (4, 5),
 (4, 6),
 (5, 3),
 (5, 4),
 (5, 5),
 (6, 3),
 (6, 4),
 (7, 2),
 (7, 3)}

validation
(2,7) is the same as (7,2)
(3,6) the same as (6,3)
(4,6) same as (6,4)
(5,5) symmetric

test
(3,7) the same as (7,3)
(4,4) symmetric
(3,5) the same as (5,3)
(4,5) same as (5,4)

In [70]:
training_num_pairs = [[x[0],x[1]] for x in already_found_in_training_add_reverses]

In [71]:
validation_num_pairs = [[2,7],[7,2],[3,6],[6,3],[4,6],[6,4],[5,5]]

In [72]:
test_num_pairs = [[3,7],[7,3],[4,4],[3,5],[5,3],[4,5],[5,4]]

In [73]:
len(training_num_pairs),len(validation_num_pairs),len(test_num_pairs)

(14, 7, 7)

In [16]:
from collections import defaultdict
def generate_sum_dic(more_in_hypothesis):
    dic = defaultdict(list)
    for target in range(4, 11):
        tmp = []
        for i in range(2, target - 1):
            tmp.append([i, target - i])
            
        if more_in_hypothesis:
            for x in range(target+1, 11):  
                dic[x].extend(tmp)
        else:
            dic[target].extend(tmp)
    return dic

entailment_sum_dict = generate_sum_dic(False)
neutral_sum_dict = generate_sum_dic(True)

entailment_sum_dict, neutral_sum_dict

(defaultdict(list,
             {4: [[2, 2]],
              5: [[2, 3], [3, 2]],
              6: [[2, 4], [3, 3], [4, 2]],
              7: [[2, 5], [3, 4], [4, 3], [5, 2]],
              8: [[2, 6], [3, 5], [4, 4], [5, 3], [6, 2]],
              9: [[2, 7], [3, 6], [4, 5], [5, 4], [6, 3], [7, 2]],
              10: [[2, 8], [3, 7], [4, 6], [5, 5], [6, 4], [7, 3], [8, 2]]}),
 defaultdict(list,
             {5: [[2, 2]],
              6: [[2, 2], [2, 3], [3, 2]],
              7: [[2, 2], [2, 3], [3, 2], [2, 4], [3, 3], [4, 2]],
              8: [[2, 2],
               [2, 3],
               [3, 2],
               [2, 4],
               [3, 3],
               [4, 2],
               [2, 5],
               [3, 4],
               [4, 3],
               [5, 2]],
              9: [[2, 2],
               [2, 3],
               [3, 2],
               [2, 4],
               [3, 3],
               [4, 2],
               [2, 5],
               [3, 4],
               [4, 3],
               [5, 2]

In [74]:
from collections import defaultdict
def generate_sum_dic(more_in_hypothesis, include_indices):
    dic = defaultdict(list)
    for target in range(4, 11):
        tmp = []
        for i in range(2, target - 1):
            if [i, target - i] in include_indices:
                tmp.append([i, target - i])
        if more_in_hypothesis:
            for x in range(target+1, 11):  
                dic[x].extend(tmp)
        else:
            dic[target].extend(tmp)
    return dic

entailment_sum_dict_train = generate_sum_dic(False, training_num_pairs)
neutral_sum_dict_train = generate_sum_dic(True, training_num_pairs)
entailment_sum_dict_val = generate_sum_dic(False, validation_num_pairs)
neutral_sum_dict_val = generate_sum_dic(True, validation_num_pairs)
entailment_sum_dict_test = generate_sum_dic(False, test_num_pairs)
neutral_sum_dict_test = generate_sum_dic(True, test_num_pairs)

In [76]:
entailment_sum_dict_train

defaultdict(list,
            {4: [[2, 2]],
             5: [[2, 3], [3, 2]],
             6: [[2, 4], [3, 3], [4, 2]],
             7: [[2, 5], [3, 4], [4, 3], [5, 2]],
             8: [[2, 6], [6, 2]],
             9: [],
             10: [[2, 8], [8, 2]]})

In [77]:
entailment_sum_dict_val

defaultdict(list,
            {4: [],
             5: [],
             6: [],
             7: [],
             8: [],
             9: [[2, 7], [3, 6], [6, 3], [7, 2]],
             10: [[4, 6], [5, 5], [6, 4]]})

In [78]:
entailment_sum_dict_test

defaultdict(list,
            {4: [],
             5: [],
             6: [],
             7: [],
             8: [[3, 5], [4, 4], [5, 3]],
             9: [[4, 5], [5, 4]],
             10: [[3, 7], [7, 3]]})

In [17]:
def generate_sum_dic_multi(sum_dic):
    sum_dict_multi = dict()
    for target in sum_dic:
        tmp = []
        pairs = sum_dic[target]
        for pair in pairs:
            tmp.extend(utils.num_pair_to_4_possible_mix(pair)) 
        sum_dict_multi[target] = tmp
    return sum_dict_multi

In [79]:
entailment_sum_dict_multi_train = generate_sum_dic_multi(entailment_sum_dict_train)
neutral_sum_dict_multi_train = generate_sum_dic_multi(neutral_sum_dict_train)
entailment_sum_dict_multi_val = generate_sum_dic_multi(entailment_sum_dict_val)
neutral_sum_dict_multi_val = generate_sum_dic_multi(neutral_sum_dict_val)
entailment_sum_dict_multi_test = generate_sum_dic_multi(entailment_sum_dict_test)
neutral_sum_dict_multi_test = generate_sum_dic_multi(neutral_sum_dict_test)

In [80]:
neutral_sum_dict_multi_val

{5: [],
 6: [],
 7: [],
 8: [],
 9: [],
 10: [('2', '7'),
  ('2', 'seven'),
  ('two', '7'),
  ('two', 'seven'),
  ('3', '6'),
  ('3', 'six'),
  ('three', '6'),
  ('three', 'six'),
  ('6', '3'),
  ('6', 'three'),
  ('six', '3'),
  ('six', 'three'),
  ('7', '2'),
  ('7', 'two'),
  ('seven', '2'),
  ('seven', 'two')]}

In [83]:
def generate_equal_sum_additions(binary_list, more_in_hypothesis, entailment_sum_dict_multi, neutral_sum_dict_multi):
    res = []
    for i in range(len(binary_list)):
        orig_rem_bin_pair = utils.remove_parenthesis(binary_list[i])
        num_pairs = numerical_word_in_sentence(binary_list[i][0]), numerical_word_in_sentence(binary_list[i][1])

        # If A + B = C, replace the three numbers
        if sum_is_equal(num_pairs):
            if more_in_hypothesis:
                dic = neutral_sum_dict_multi
            else:
                dic = entailment_sum_dict_multi
                
            for key in dic:
                # C=key, (A, B) = pair
                for pair in dic[key]:
                    sent1_binary = binary_list[i][0]   
                    # TODO: replacing the first word in a sentence with a new word breaks capitalization.
                    sent1_binary = utils.return_sent_words_replaced_case_sensitive(sent1_binary, num_pairs[0][0], pair[0]) # replace A
                    sent1_binary = utils.return_sent_words_replaced_case_sensitive(sent1_binary, num_pairs[0][1], pair[1]) # replace B
                    
                    # Create both '7' and 'seven' for C
                    for num in (str(key), utils.reverse_convert_dict[key]):
                        tmp_dict = dict()
                        sent2_binary = binary_list[i][1]
                        sent2_binary = utils.return_sent_words_replaced_case_sensitive(sent2_binary, num_pairs[1][0], num) # replace C
                        rem_parenthesis_pair = utils.remove_parenthesis((sent1_binary, sent2_binary))
                        
                        tmp_dict['gold_label'] = 'neutral' if more_in_hypothesis else 'entailment'
                        tmp_dict['sentence1'] = rem_parenthesis_pair[0]
                        tmp_dict['sentence2'] = rem_parenthesis_pair[1]
                        tmp_dict['sentence1_binary_parse'] = sent1_binary
                        tmp_dict['sentence2_binary_parse'] = sent2_binary
                        tmp_dict['orig_sentence1'] = orig_rem_bin_pair[0]
                        tmp_dict['orig_sentence2'] = orig_rem_bin_pair[1]
                        tmp_dict['orig_sentence1_binary_parse'] = binary_list[i][0]
                        tmp_dict['orig_sentence2_binary_parse'] = binary_list[i][1]   
                        
                        res.append(tmp_dict)
                        
    return res
        

In [84]:
test_additions_entailment = generate_equal_sum_additions(test_bin_ent_two2one, False, entailment_sum_dict_multi_test, neutral_sum_dict_multi_test)
test_additions_neutral = generate_equal_sum_additions(test_bin_ent_two2one, True, entailment_sum_dict_multi_test, neutral_sum_dict_multi_test)

train_additions_entailment = generate_equal_sum_additions(train_bin_ent_two2one, False, entailment_sum_dict_multi_train, neutral_sum_dict_multi_train)
train_additions_neutral = generate_equal_sum_additions(train_bin_ent_two2one, True, entailment_sum_dict_multi_train, neutral_sum_dict_multi_train)

dev_additions_entailment = generate_equal_sum_additions(dev_bin_ent_two2one, False, entailment_sum_dict_multi_val, neutral_sum_dict_multi_val)
dev_additions_neutral = generate_equal_sum_additions(dev_bin_ent_two2one, True, entailment_sum_dict_multi_val, neutral_sum_dict_multi_val)

In [85]:
test_additions_entailment[:20:4]

[{'gold_label': 'entailment',
  'sentence1': '3 women and 5 girls crossing the road',
  'sentence2': 'There are 8 female humans who are moving .',
  'sentence1_binary_parse': '( ( ( 3 women ) and ) ( ( 5 girls ) ( crossing ( the road ) ) ) )',
  'sentence2_binary_parse': '( There ( ( are ( ( 8 ( female humans ) ) ( who ( are moving ) ) ) ) . ) )',
  'orig_sentence1': '2 women and 2 girls crossing the road',
  'orig_sentence2': 'There are 4 female humans who are moving .',
  'orig_sentence1_binary_parse': '( ( ( 2 women ) and ) ( ( 2 girls ) ( crossing ( the road ) ) ) )',
  'orig_sentence2_binary_parse': '( There ( ( are ( ( 4 ( female humans ) ) ( who ( are moving ) ) ) ) . ) )'},
 {'gold_label': 'entailment',
  'sentence1': 'three women and 5 girls crossing the road',
  'sentence2': 'There are 8 female humans who are moving .',
  'sentence1_binary_parse': '( ( ( three women ) and ) ( ( 5 girls ) ( crossing ( the road ) ) ) )',
  'sentence2_binary_parse': '( There ( ( are ( ( 8 ( fema

In [22]:
test_additions_neutral[:20:4]

[{'gold_label': 'neutral',
  'sentence1': '2 women and 2 girls crossing the road',
  'sentence2': 'There are 5 female humans who are moving .',
  'sentence1_binary_parse': '( ( ( 2 women ) and ) ( ( 2 girls ) ( crossing ( the road ) ) ) )',
  'sentence2_binary_parse': '( There ( ( are ( ( 5 ( female humans ) ) ( who ( are moving ) ) ) ) . ) )',
  'orig_sentence1': '2 women and 2 girls crossing the road',
  'orig_sentence2': 'There are 4 female humans who are moving .',
  'orig_sentence1_binary_parse': '( ( ( 2 women ) and ) ( ( 2 girls ) ( crossing ( the road ) ) ) )',
  'orig_sentence2_binary_parse': '( There ( ( are ( ( 4 ( female humans ) ) ( who ( are moving ) ) ) ) . ) )'},
 {'gold_label': 'neutral',
  'sentence1': 'two women and 2 girls crossing the road',
  'sentence2': 'There are 5 female humans who are moving .',
  'sentence1_binary_parse': '( ( ( two women ) and ) ( ( 2 girls ) ( crossing ( the road ) ) ) )',
  'sentence2_binary_parse': '( There ( ( are ( ( 5 ( female humans 

## 2. Reverse a contradiction to get a contradiction

In [94]:
test_data_cont = subset_by_relationship_type(test_data, 'contradiction')
train_data_cont = subset_by_relationship_type(train_data, 'contradiction')
dev_data_cont = subset_by_relationship_type(dev_data, 'contradiction')

In [95]:
test_cont_with_numericals = subset_with_numericals(test_data_cont)
train_cont_with_numericals = subset_with_numericals(train_data_cont)
dev_cont_with_numericals = subset_with_numericals(dev_data_cont)

In [88]:
def multiple2one(dataset):
    result_dic = []
    num_word_list1 = []
    num_word_list2 = []
    for data in dataset:
        sent1_num_words = numerical_word_in_sentence(data['sentence1'])
        sent2_num_words = numerical_word_in_sentence(data['sentence2'])

        if len(sent1_num_words) > 1 and len(sent2_num_words) == 1:
            print((sent1_num_words, sent2_num_words))
            print(data['gold_label'])
            print(data['sentence1'])
            print(data['sentence2'])
            print()
            result_dic.append(data)
            num_word_list1.append(sent1_num_words)
            num_word_list2.append(sent2_num_words)
    return result_dic, num_word_list1, num_word_list2

In [96]:
test_dic, n1, n2 = multiple2one(test_cont_with_numericals)
train_dic, train_n1, train_n2 = multiple2one(train_cont_with_numericals)
dev_dic, dev_n1, dev_n2 = multiple2one(dev_cont_with_numericals)

(['Five', 'two'], ['three'])
contradiction
Five girls and two guys are crossing a overpass.
The three men sit and talk about their lives.

(['2', '2'], ['Four'])
contradiction
2 women and 2 girls crossing the road
Four male humans crossing the road.

(['Four', 'two', 'two'], ['four'])
contradiction
Four men are sitting in front of two large windows; one man is playing a guitar, one is playing the bongo drums, and two are sitting back without instruments.
The four men are playing hockey.

(['Two', 'four'], ['Six'])
contradiction
Two men stand in a lift as four others, all dressed in black and bright yellow stripes, look up on them.
Six men are seated at the back of a city bus.

(['Two', 'two'], ['Four'])
contradiction
Two women, standing at a fence, are each talking on their cellphones, while two men sitting on the opposite side of the fence are having a conversation.
Four people are talking to each other outside.

(['Two', 'two'], ['two'])
contradiction
Two women, standing at a fence, 


(['two', 'three'], ['two'])
contradiction
There are two small boys wearing ball caps, playing T-Ball in green grass with three adult men watching.
Men play T-Ball while two boys watch.

(['two', 'three'], ['Two'])
contradiction
There are two small boys wearing ball caps, playing T-Ball in green grass with three adult men watching.
Two boys are playing a game of soccer by themselves outside.

(['two', 'three'], ['3'])
contradiction
There are two small boys wearing ball caps, playing T-Ball in green grass with three adult men watching.
There are 3 men watching girls play t-ball in a field.

(['two', 'three'], ['two'])
contradiction
A family of two woman and a young child sits on the grass in a park in front of three young girls.
A family of two men and a boy sits on the roof of a building

(['two', 'three'], ['Two'])
contradiction
A family of two woman and a young child sits on the grass in a park in front of three young girls.
Two women are having a picnic and are eating babies.

(['ni

There are five men.

(['Three', 'two'], ['Three'])
contradiction
Three younger heavyset ladies, dressed nicely in warm weather clothing adjusting themselves while walking down a sidewalk with palm trees in the background and two younger men following several feet behind.
Three curvy ladies dressed nicely for the weather were walking down a sidewalk with palm trees to get away from the men that were chasing them down.

(['Six', 'two'], ['3'])
contradiction
Six children are sitting in two rows with plastic bags on the desk in front of them.
3 children are walking around with plastic bags.

(['Six', 'two'], ['six'])
contradiction
Six African children sitting at two tables.
The six African children are watching television.

(['Two', 'two'], ['Two'])
contradiction
Two people dressed in athletic clothing walk down the street, while the taller of the two is holding a basketball.
Two people are about to play nintendo.

(['Two', 'two'], ['Two'])
contradiction
Two people dressed in athletic clot

(['Three', 'two'], ['three'])
contradiction
Three grown adults in shorts attempting to practice yoga, two adults are wearing tank tops, while the other is shirtless.
The three adults are all wearing dresses.

(['Three', 'two'], ['Three'])
contradiction
Three grown adults in shorts attempting to practice yoga, two adults are wearing tank tops, while the other is shirtless.
Three adults are sleeping.

(['four', 'four'], ['Four'])
contradiction
A group of four males performing in front of an audience with four lights projecting from round holes in the background
Four men are sitting in the shade under the bright glare of the sun

(['Three', 'two'], ['Three'])
contradiction
Three men are pictured, two standing and one sitting in front of silver container.
Three ladies sit around a blue container.

(['Three', 'three'], ['two'])
contradiction
Three on three table tennis between young and older.
The two professional ping-pong players were in the final round of the world championships.

(['Sev

In [90]:
n2

[['three'],
 ['Four'],
 ['four'],
 ['Six'],
 ['Four'],
 ['two'],
 ['Six'],
 ['Three'],
 ['two'],
 ['Three'],
 ['Six'],
 ['Two'],
 ['three'],
 ['3']]

In [91]:
def reverse_format(num_string):
    num_string = num_string.lower()
    if num_string in utils.reverse_convert_dict.values():
        return str(utils.convert_dict[num_string])
    return utils.reverse_convert_dict[int(num_string)]

# 3. Use augmented entailment to generate augmented contradiction

In [97]:
import spacy
nlp = spacy.load('en')

def nummod_in_sentence(sentence):
    doc = nlp(sentence.lower())
    return [[token.text, token.head.text] for token in doc 
            if token.text in convert_dict_plural and token.dep_=='nummod']

def check_hyponym(hypo_word, word_h):
    #print(hypo_word, word)
    word0 = wn.synsets(hypo_word)
    word = wn.synsets(word_h)
    if len(word0)>= 1 and len(word)>=1:
        word0 = word0[0]
        word = word[0]
        hypowords = set([i for i in word.closure(lambda s:s.hyponyms())])
        return word0 in hypowords
    else:
        print("{} or {} is not found in wordnet".format(hypo_word, word_h))
        return False
# Find the word to replace to an antonym word. Pattern: the word after numerical word
def return_next_word(word, sentence):
    list_of_words = sentence.split()
    if list_of_words.index(word) + 1 < len(list_of_words):
        return list_of_words[list_of_words.index(word) + 1]
    else:
        raise Exception('{} is the last word in the sentence'.format(word))

noun_to_replace = return_next_word("two", "There are two boys")
print(noun_to_replace)

from nltk.corpus import wordnet
#from pattern.en import pluralize
import inflect

# Return the antonym of input word. Make them plural # TODO: pluralize doesn't make sense when the antonym is an adjective
def return_antonym_plural(word):
    engine = inflect.engine()
    antonyms = []
    for syn in wordnet.synsets(word):
        for lm in syn.lemmas():
            if lm.antonyms():
                antonyms.append(lm.antonyms()[0].name())        
    if antonyms:
        antonyms.sort(key=lambda item: (len(item), item))
        #return [' '.join(engine.plural(w).split('_')) for w in list(set(antonyms))]
        return engine.plural(antonyms[0])
        #return pluralize(antonyms[0])
    else:
        print('Antonym of {} could not be found.'.format(word))
        return None

return_antonym_plural('boys')

boys


'girls'

In [100]:
def generate_same_to_different_plural_change_s2(binary_list):
    res = []
    for i in range(len(binary_list)):
        sentence1_bin = binary_list[i][0]
        sentence2_bin = binary_list[i][1]
        sentence1, sentence2 = utils.remove_parenthesis(binary_list[i])
        nummods = nummod_in_sentence(sentence2)
        if len(nummods)==0:
            print('no nummod in', sentence2)
            continue
        #noun_to_replace = return_next_word(numerical_word, sentence2) # subject to be replaced in sentence2
        #ant_noun_to_replace = return_antonym_plural(noun_to_replace) # antonym of the subject
        ant_noun_to_replace = return_antonym_plural(nummods[0][1])
        if ant_noun_to_replace:
            try:
                new_sentence2_bin = utils.return_sent_words_replaced_case_sensitive(
                    sentence2_bin, 
                    nummods[0][1],#noun_to_replace, 
                    ant_noun_to_replace
                )
                new_sentence2 = utils.return_sent_words_replaced_case_sensitive(
                    sentence2, 
                    nummods[0][1],#noun_to_replace, 
                    ant_noun_to_replace
                )            

                tmp_dict = dict()
                tmp_dict['gold_label'] = 'contradiction'
                tmp_dict['sentence1'] = sentence1
                tmp_dict['sentence2'] = new_sentence2
                tmp_dict['sentence1_binary_parse'] = binary_list[i][0]
                tmp_dict['sentence2_binary_parse'] = new_sentence2_bin
                tmp_dict['orig_sentence1'] = sentence1
                tmp_dict['orig_sentence2'] = sentence2
                tmp_dict['orig_sentence1_binary_parse'] = binary_list[i][0]
                tmp_dict['orig_sentence2_binary_parse'] = binary_list[i][1]  
                res.append(tmp_dict)
            except:
                pass
        
    return res

In [98]:
test_entailment_augumented = get_binary_pairs(test_additions_entailment) 
train_entailment_augumented = get_binary_pairs(train_additions_entailment) 
dev_entailment_augumented = get_binary_pairs(dev_additions_entailment) 

In [101]:
test_contradiction_subject2_to_antonym = generate_same_to_different_plural_change_s2(test_entailment_augumented)
dev_contradiction_subject2_to_antonym = generate_same_to_different_plural_change_s2(dev_entailment_augumented)

Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of humans could not be found.
Antonym of h

Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.


In [102]:
#test_contradiction_subject2_to_antonym = generate_same_to_different_plural_change_s2(test_entailment_augumented)
train_contradiction_subject2_to_antonym = generate_same_to_different_plural_change_s2(train_entailment_augumented)
#dev_contradiction_subject2_to_antonym = generate_same_to_different_plural_change_s2(dev_entailment_augumented)

Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could not be found.
Antonym of players could 

Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of p

Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of p

Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of p

Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could not be found.
Antonym of mammals could 

...

Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of p

Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of p

Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of p

no nummod in A group of seven is walking down a hallway .
no nummod in A group of 7 is walking down a hallway .
no nummod in A group of seven is walking down a hallway .
no nummod in A group of 7 is walking down a hallway .
no nummod in A group of seven is walking down a hallway .
no nummod in A group of 7 is walking down a hallway .
no nummod in A group of seven is walking down a hallway .
no nummod in A group of 7 is walking down a hallway .
no nummod in A group of seven is walking down a hallway .
no nummod in A group of 7 is walking down a hallway .
no nummod in A group of seven is walking down a hallway .
no nummod in A group of 7 is walking down a hallway .
no nummod in A group of seven is walking down a hallway .
no nummod in A group of 7 is walking down a hallway .
no nummod in A group of seven is walking down a hallway .
no nummod in A group of 7 is walking down a hallway .
no nummod in A group of seven is walking down a hallway .
no nummod in A group of 7 is walking down a ha

Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of people could not be found.
Antonym of p

In [105]:
len(test_additions_entailment),len(test_additions_neutral),len(test_dic),len(test_contradiction_subject2_to_antonym)

(168, 192, 14, 56)

In [104]:
len(train_additions_entailment),len(train_additions_neutral),len(train_dic),len(train_contradiction_subject2_to_antonym)

(11760, 36960, 616, 3584)

In [106]:
len(dev_additions_entailment),len(dev_additions_neutral),len(dev_dic),len(dev_contradiction_subject2_to_antonym)

(168, 96, 20, 56)

In [108]:
import jsonlines
with jsonlines.open('augmented_addition_train_split_pairs.jsonl', mode='w') as writer:
    writer.write_all(train_additions_entailment*3+train_additions_neutral+(train_dic+train_contradiction_subject2_to_antonym)*9)
with jsonlines.open('augmented_addition_dev_split_pairs.jsonl', mode='w') as writer:
    writer.write_all(dev_additions_entailment+dev_additions_neutral+dev_dic+dev_contradiction_subject2_to_antonym)
with jsonlines.open('augmented_addition_test_split_pairs.jsonl', mode='w') as writer:
    writer.write_all(test_additions_entailment+test_additions_neutral+test_dic+test_contradiction_subject2_to_antonym)
with jsonlines.open('augmented_addition_test_entailment_split_pairs.jsonl', mode='w') as writer:
    writer.write_all(test_additions_entailment)
with jsonlines.open('augmented_addition_test_neutral_split_pairs.jsonl', mode='w') as writer:
    writer.write_all(test_additions_neutral)
with jsonlines.open('augmented_addition_test_contradiction_split_pairs.jsonl', mode='w') as writer:
    writer.write_all(test_dic+test_contradiction_subject2_to_antonym)

In [110]:
def save_examples_for_ESIM(filename, example_dict_list):
    global_pair_index = 0
    with open(filename,'w+') as f:
        for example_dict in example_dict_list:
            f.write("{}\t{}\t{}\t3\t4\t5\t6\t{}\n".format(
                example_dict['gold_label'],
                example_dict['sentence1_binary_parse'],
                example_dict['sentence2_binary_parse'],
                global_pair_index
            ))
            global_pair_index += 1
                

In [111]:
save_examples_for_ESIM('augmented_addition_train_split_pairs.tsv', train_additions_entailment*3+train_additions_neutral+(train_dic+train_contradiction_subject2_to_antonym)*9)
save_examples_for_ESIM('augmented_addition_dev_split_pairs.tsv', dev_additions_entailment+dev_additions_neutral+dev_dic+dev_contradiction_subject2_to_antonym)
save_examples_for_ESIM('augmented_addition_test_split_pairs.tsv', test_additions_entailment+test_additions_neutral+test_dic+test_contradiction_subject2_to_antonym)
save_examples_for_ESIM('augmented_addition_test_entailment_split_pairs.tsv', test_additions_entailment)
save_examples_for_ESIM('augmented_addition_test_neutral_split_pairs.tsv',  test_additions_neutral)
save_examples_for_ESIM('augmented_addition_test_contradiction_split_pairs.tsv', test_dic+test_contradiction_subject2_to_antonym)